# Latent Semantic Analysis

# Data Crawling

```python
import scrapy

class BlogSpider(scrapy.Spider):
    name = 'blog'
    allowed_domains = ['pta.trunojoyo.ac.id']
    start_urls = ['https://pta.trunojoyo.ac.id/c_search/byprod/10']

    def __init__(self, name=None, **kwargs):
        kwargs['include_headers_line'] = False
        self.count = 0
        super().__init__(name, **kwargs)

    def parse(self, response):
        for url in response.css('a.gray.button'):
            yield response.follow(url.attrib['href'], self.parse_item)
        if self.count < 12:
            self.count += 1
            yield response.follow(response.xpath("//*[@class='pag_button' and contains(text(), '>')]").attrib['href'], self.parse)

    def parse_item(self, response):
        yield {
            "judul": response.css('a.title::text').get(),
            "penulis": response.xpath('//*[@id="content_journal"]/ul/li/div[2]/div[1]/span/text()').get()[10:],
            "dospem1": response.xpath('//*[@id="content_journal"]/ul/li/div[2]/div[2]/span/text()').get()[21:],
            "dospem2": response.xpath('//*[@id="content_journal"]/ul/li/div[2]/div[3]/span/text()').get()[21:],
            "abstrak": response.xpath('//*[@id="content_journal"]/ul/li/div[4]/div[2]/p/text()').get(),
        }
```

# Imports
Pertama-tama kita membutuhkan library yang akan dipakai untuk pemrosesan ini, diantaranya adalah:

- [x] numpy
- [x] regex
- [x] pandas
- [x] nltk
- [x] sklearn
- [x] sastrawi

In [18]:
import regex as re
import pandas as pd
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/zaka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/zaka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Global Vars

| Variable   | Reference       | Library  |
| ---------- | --------------- | -------- |
| factory    | StemmerFactory  | Sastrawi |
| vectorizer | CountVectorizer | sklearn  |factory_dictionary
| tfidf      | TfidfVectorizer | sklearn  |
| idf        | TfidfVectorizer | sklearn  |

In [19]:
factory = StemmerFactory()
vectorizer = CountVectorizer()
tfidf = TfidfVectorizer()
idf = TfidfVectorizer(use_idf=True)
factory_dictionary = factory.get_words()
stemmer = factory.create_stemmer()
indo_stopwords = stopwords.words('indonesian')
indo_stopwords.extend(stopwords.words('english'))
docs = pd.read_csv('data.csv', usecols=['abstrak'])
docs = docs.rename(columns={'abstrak': 'raw'})
docs

,raw
0,ABSTRAK\nAkhir-akhir ini semakin maraknya penc...
1,Makanan dan minuman khas merupakan ciri dari k...
2,Penelitian ini bertujuan untuk dapat mengetahu...
3,Abstrak\n\n Rian Pandi Pranata. NIM 1605211000...
4,Perairan Soccah merupakan lokasi nelayan menca...
...,...
158,ABSTRAK\nANALISIS STRATEGI PEMASARAN DENGAN FI...
159,Berita merupakan pesan atau informasi hasil da...
160,Penelitian ini dilakukan untuk menjawab permas...
161,Tujuan dari penelitian ini adalah untuk menemu...


# Cleaning & Tokenizing
Data yang sudah dicrawling akan dibersihkan dan akan di _tokenisasi_. data dibersihkan dengan membuang _punctuation_. kemudian _punctuation_ tersebut akan diganti dengan spasi

##### Punctuation Table
| Name             | Character | Example |
| ---------------- | --------- | ------- |
| Full Stop        | .         | .       |
| Comma            | ,         | ,       |
| Question Mark    | ?         | ?       |
| Exclamation Mark | !         | !       |
| Quotation Marks  | " "       | "word"  |
| Apestrophe       | `         | `       |
| Hypen            | -         | -       |
| Dash             | -         | -or--   |
| Colon            | :         | :       |
| Semicolon        | ;         | ;       |
| Parantheses      | ()        | ()      |
| Brackets         | []        | []      |
| Ellipsis         | ...       | ...     |
| Slash            | /         | /       |

In [20]:
docs['text'] = docs['raw'].apply(
    lambda x: word_tokenize(re.sub('[^a-zA-Z]', ' ', str(x).lower())))
docs

,raw,text
0,ABSTRAK\nAkhir-akhir ini semakin maraknya penc...,"[abstrak, akhir, akhir, ini, semakin, maraknya..."
1,Makanan dan minuman khas merupakan ciri dari k...,"[makanan, dan, minuman, khas, merupakan, ciri,..."
2,Penelitian ini bertujuan untuk dapat mengetahu...,"[penelitian, ini, bertujuan, untuk, dapat, men..."
3,Abstrak\n\n Rian Pandi Pranata. NIM 1605211000...,"[abstrak, rian, pandi, pranata, nim, program, ..."
4,Perairan Soccah merupakan lokasi nelayan menca...,"[perairan, soccah, merupakan, lokasi, nelayan,..."
...,...,...
158,ABSTRAK\nANALISIS STRATEGI PEMASARAN DENGAN FI...,"[abstrak, analisis, strategi, pemasaran, denga..."
159,Berita merupakan pesan atau informasi hasil da...,"[berita, merupakan, pesan, atau, informasi, ha..."
160,Penelitian ini dilakukan untuk menjawab permas...,"[penelitian, ini, dilakukan, untuk, menjawab, ..."
161,Tujuan dari penelitian ini adalah untuk menemu...,"[tujuan, dari, penelitian, ini, adalah, untuk,..."


# Stopwords
Didalam _text mining_ terdapat _stopwords_, dimana itu adalah kumpulan kosa kata yang tidak mengandung arti sehingga tidak diperlukan untuk proses _mining_ nanti. Jika kata terdeteksi sebagai _stopwords_ maka kata tersebut akan dihapus dari sebuah dokumen

#### Stopwords Dictionary
<a href="https://gist.github.com/sebleier/554280" title="NLTK's list of english stopwords">English Stopwords Dictionary</a>

<a href="https://github.com/stopwords-iso/stopwords-id/blob/master/raw/indonesian-stopwords-complete.txt" title="indonesian-stopwords-complete.txt">Indonesian Stopwords Dictionary</a>

> Kedua _stopwords_ tadi digabung menjadi satu

In [21]:
docs['text'] = docs['text'].apply(
    lambda doc: [d for d in doc if d not in indo_stopwords])
    # lambda doc: [d for d in doc if d not in indo_stopwords and d in factory_dictionary])
docs

,raw,text
0,ABSTRAK\nAkhir-akhir ini semakin maraknya penc...,"[abstrak, maraknya, pencurian, kayu, milik, ne..."
1,Makanan dan minuman khas merupakan ciri dari k...,"[makanan, minuman, khas, ciri, kekayaan, sumbe..."
2,Penelitian ini bertujuan untuk dapat mengetahu...,"[penelitian, bertujuan, pengaruh, motivasi, ke..."
3,Abstrak\n\n Rian Pandi Pranata. NIM 1605211000...,"[abstrak, rian, pandi, pranata, nim, program, ..."
4,Perairan Soccah merupakan lokasi nelayan menca...,"[perairan, soccah, lokasi, nelayan, mencari, i..."
...,...,...
158,ABSTRAK\nANALISIS STRATEGI PEMASARAN DENGAN FI...,"[abstrak, analisis, strategi, pemasaran, finte..."
159,Berita merupakan pesan atau informasi hasil da...,"[berita, pesan, informasi, hasil, proses, kons..."
160,Penelitian ini dilakukan untuk menjawab permas...,"[penelitian, permasalahan, praktik, hadiah, ku..."
161,Tujuan dari penelitian ini adalah untuk menemu...,"[tujuan, penelitian, menemukan, pengaruh, peng..."


# Stemming
Didalam _text mining_ terdapat _stemming_, dimana itu adalah sebuah cara untuk mentransformasikan sebuha kata menjadi bentuk pertama atau menjadi bentuk akarnya, seperti:

memakan menjadi makan

#### Stemming Dictionary
<a href="https://github.com/har07/PySastrawi/blob/master/src/Sastrawi/Stemmer/data/kata-dasar.txt" title="kata-dasar.txt">Indonesian Stemmer Dictionary</a>

In [22]:
term_dict = {}

for doc in docs['text']:
    for term in doc:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmer.stem(term)

docs['text'] = docs['text'].apply(
    lambda doc: [term_dict[term] for term in doc])
docs

,raw,text
0,ABSTRAK\nAkhir-akhir ini semakin maraknya penc...,"[abstrak, marak, curi, kayu, milik, negara, hu..."
1,Makanan dan minuman khas merupakan ciri dari k...,"[makan, minum, khas, ciri, kaya, sumberdaya, a..."
2,Penelitian ini bertujuan untuk dapat mengetahu...,"[teliti, tuju, pengaruh, motivasi, kerja, medi..."
3,Abstrak\n\n Rian Pandi Pranata. NIM 1605211000...,"[abstrak, rian, pandi, pranata, nim, program, ..."
4,Perairan Soccah merupakan lokasi nelayan menca...,"[air, soccah, lokasi, nelayan, cari, ikan, org..."
...,...,...
158,ABSTRAK\nANALISIS STRATEGI PEMASARAN DENGAN FI...,"[abstrak, analisis, strategi, pasar, fintech, ..."
159,Berita merupakan pesan atau informasi hasil da...,"[berita, pesan, informasi, hasil, proses, kons..."
160,Penelitian ini dilakukan untuk menjawab permas...,"[teliti, masalah, praktik, hadiah, kupon, undi..."
161,Tujuan dari penelitian ini adalah untuk menemu...,"[tuju, teliti, temu, pengaruh, alam, sesuai, t..."


# Frequency Distribution
Dipakai untuk menghitung distribusi frekuensi dari setiap kata

In [23]:
docs['freq'] = docs['text'].apply(FreqDist)
docs

,raw,text,freq
0,ABSTRAK\nAkhir-akhir ini semakin maraknya penc...,"[abstrak, marak, curi, kayu, milik, negara, hu...","{'abstrak': 1, 'marak': 1, 'curi': 4, 'kayu': ..."
1,Makanan dan minuman khas merupakan ciri dari k...,"[makan, minum, khas, ciri, kaya, sumberdaya, a...","{'makan': 8, 'minum': 9, 'khas': 9, 'ciri': 1,..."
2,Penelitian ini bertujuan untuk dapat mengetahu...,"[teliti, tuju, pengaruh, motivasi, kerja, medi...","{'teliti': 4, 'tuju': 1, 'pengaruh': 2, 'motiv..."
3,Abstrak\n\n Rian Pandi Pranata. NIM 1605211000...,"[abstrak, rian, pandi, pranata, nim, program, ...","{'abstrak': 1, 'rian': 1, 'pandi': 1, 'pranata..."
4,Perairan Soccah merupakan lokasi nelayan menca...,"[air, soccah, lokasi, nelayan, cari, ikan, org...","{'air': 7, 'soccah': 2, 'lokasi': 1, 'nelayan'..."
...,...,...,...
158,ABSTRAK\nANALISIS STRATEGI PEMASARAN DENGAN FI...,"[abstrak, analisis, strategi, pasar, fintech, ...","{'abstrak': 1, 'analisis': 1, 'strategi': 5, '..."
159,Berita merupakan pesan atau informasi hasil da...,"[berita, pesan, informasi, hasil, proses, kons...","{'berita': 11, 'pesan': 1, 'informasi': 2, 'ha..."
160,Penelitian ini dilakukan untuk menjawab permas...,"[teliti, masalah, praktik, hadiah, kupon, undi...","{'teliti': 8, 'masalah': 1, 'praktik': 1, 'had..."
161,Tujuan dari penelitian ini adalah untuk menemu...,"[tuju, teliti, temu, pengaruh, alam, sesuai, t...","{'tuju': 1, 'teliti': 6, 'temu': 2, 'pengaruh'..."


#### Text Frequency Table
Agar memudahkan kita untuk melihat distribusi frekuensi dari teks, kita bisa memanfaatkan display table dengan cara memasukkan data distribusi frekuensi ke dalam _DataFrame_ baru

In [24]:
tf_table = docs['freq']
tf_table = pd.DataFrame(tf_table.tolist()).fillna(0)
# bentuk aljabar
tf_table

,abstrak,marak,curi,kayu,milik,negara,hutan,masyarakat,rumus,sanksi,...,saring,trauma,interpersonal,cemas,derita,faham,notabennya,diajarakan,slovin,ho
0,1.0,1.0,4.0,4.0,3.0,2.0,5.0,4.0,4.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [25]:
docs['text'] = docs['text'].apply(lambda doc: " ".join([x for x in doc]))
docs

,raw,text,freq
0,ABSTRAK\nAkhir-akhir ini semakin maraknya penc...,abstrak marak curi kayu milik negara hutan mas...,"{'abstrak': 1, 'marak': 1, 'curi': 4, 'kayu': ..."
1,Makanan dan minuman khas merupakan ciri dari k...,makan minum khas ciri kaya sumberdaya alam dae...,"{'makan': 8, 'minum': 9, 'khas': 9, 'ciri': 1,..."
2,Penelitian ini bertujuan untuk dapat mengetahu...,teliti tuju pengaruh motivasi kerja mediator s...,"{'teliti': 4, 'tuju': 1, 'pengaruh': 2, 'motiv..."
3,Abstrak\n\n Rian Pandi Pranata. NIM 1605211000...,abstrak rian pandi pranata nim program studi s...,"{'abstrak': 1, 'rian': 1, 'pandi': 1, 'pranata..."
4,Perairan Soccah merupakan lokasi nelayan menca...,air soccah lokasi nelayan cari ikan organisme ...,"{'air': 7, 'soccah': 2, 'lokasi': 1, 'nelayan'..."
...,...,...,...
158,ABSTRAK\nANALISIS STRATEGI PEMASARAN DENGAN FI...,abstrak analisis strategi pasar fintech btm sa...,"{'abstrak': 1, 'analisis': 1, 'strategi': 5, '..."
159,Berita merupakan pesan atau informasi hasil da...,berita pesan informasi hasil proses konstruksi...,"{'berita': 11, 'pesan': 1, 'informasi': 2, 'ha..."
160,Penelitian ini dilakukan untuk menjawab permas...,teliti masalah praktik hadiah kupon undi produ...,"{'teliti': 8, 'masalah': 1, 'praktik': 1, 'had..."
161,Tujuan dari penelitian ini adalah untuk menemu...,tuju teliti temu pengaruh alam sesuai tokoh ut...,"{'tuju': 1, 'teliti': 6, 'temu': 2, 'pengaruh'..."


# Term Frequency
Apa itu _Term Frequency_? adalah sebuah cara untuk menghitung kemunculan sebuah kata (term) dari sebuah dokumen

Formula:

    tf(t,d) = count of t in d / number of words in d
    
    where:
        t: term
        d: document

In [26]:
count_vect_res = vectorizer.fit_transform(docs['text'])
vectorizer.vocabulary_

{'abstrak': 5,
 'marak': 1658,
 'curi': 499,
 'kayu': 1258,
 'milik': 1777,
 'negara': 1891,
 'hutan': 1012,
 'masyarakat': 1678,
 'rumus': 2430,
 'sanksi': 2472,
 'pidana': 2129,
 'undang': 3056,
 'nomor': 1916,
 'cegah': 411,
 'berantas': 286,
 'rusa': 2432,
 'pasal': 2028,
 'ayat': 202,
 'jatuh': 1169,
 'laku': 1472,
 'hukum': 1006,
 'berat': 287,
 'efek': 675,
 'jera': 1175,
 'ringan': 2403,
 'tempat': 2879,
 'tinggal': 2949,
 'timbul': 2942,
 'ketidaktepatan': 1316,
 'jenis': 1172,
 'teliti': 2872,
 'tulis': 3017,
 'skripsi': 2659,
 'normatif': 1924,
 'dekat': 540,
 'pandang': 2003,
 'atur': 189,
 'objek': 1943,
 'kaji': 1207,
 'bahan': 211,
 'primer': 2210,
 'cakup': 392,
 'sekunder': 2529,
 'hasil': 960,
 'buku': 369,
 'cacat': 389,
 'tersier': 2910,
 'kamus': 1222,
 'kumpul': 1448,
 'studi': 2747,
 'pustaka': 2276,
 'konsep': 1381,
 'sejarah': 2521,
 'tindak': 2947,
 'perintah': 2101,
 'minggir': 1780,
 'hak': 938,
 'desa': 559,
 'miskin': 1787,
 'tingkat': 2951,
 'sejahtera': 

# Document Frequency
Apa itu _Document Frequency_? sekilas ini mirip seperti TF namun yang membedakan adalah ketika TF hanya menghitung banyak _term_ dalam sebuah dokumen, DF hanya menghitung berapa banyak dokumen yang mengandung sebuah _term_

Formula:

    df(t) = occurence of t in documents

    where:
        t: term

# Inverse Document Frequency
Apa itu _Invers Document Frequency_?

Ketika menghitung TF, seluruh _term_ akan dianggap sama-sama pentingnya. Meskipun begitu kata-kata seperti 'adalah', 'dari', 'itu' bisa jadi muncul beberapa kali namun tidak memiliki makna. Ditambah kita juga perlu memberikan bobot yang kecil pada _term_ dengan frekuensi yang besar dan memberikan bobot yang besar pada _term_ yang jarang muncul, dengan menghitung sebuah IDF kita bisa melakukan pembobotan ini

Formula:

    idf(t) = N/df

    where:
        t: term
        N: number of document
        df: Document Frequency

sekarang problemnya adalah ketika kita mempunyai _corpus_ yang besar (anggap 1.000.000.000), maka IDF pastinya akan lebih besar juga, untuk mengantisipasinya kita bisa menggunakan log dari sebuah idf

ketika proses perhitungannya, ketika sebuah DF bernilai 0 maka perhitungan tidak bisa dilakukan, untuk mengantisipasinya kita bisa menambahkan 1 pada DF

Final Formula:

    idf(t) = log(N/(DF+1))

    where:
        t: term
        N: number of document
        DF: Document Frequency

In [27]:
idf_res = idf.fit_transform(docs['text'])
idf.idf_

array([5.40671925, 5.40671925, 3.32727771, ..., 5.40671925, 5.40671925,
       5.40671925])

# Term Frequency Invers Document Frequency

Formula:

    tf-idf(t,d) = tf(t,d) * log(N/(DF+1))

    where:
        t: term
        d: document
        N: number of document
        DF: Document Frequency

Formula secara matematis:

$$
W_{i,j} = \frac{n_{i,j}}{\sum_{j=1}^{p} n_{j,i}} \log _{2} \frac{D}{d_{j}}
$$

keterangan:

${W_{i,j}}\quad\quad\> :$ pembobotan tf-idf untuk term ke-j pada dokumen ke-i

${n_{i,j}}\quad\quad\>\> :$ jumlah kemunculan term ke-j pada dokumen ke-i

${p} \quad\quad\quad\>\> :$ banyaknya term yang terbentuk

${\sum_{j=1}^{p} n_{j,i}} :$ jumlah kemunculan seluruh term pada dokumen ke-i

${d_{j}} \quad\quad\quad :$ banyaknya dokumen yang mengandung term ke-j

In [28]:
tfidf_res = tfidf.fit_transform(docs['text'])
print(tfidf_res)

  (0, 1449)	0.015370514606224428
  (0, 3013)	0.014602829037019295
  (0, 121)	0.019396162333877795
  (0, 521)	0.0232836746579466
  (0, 1561)	0.034539785687289735
  (0, 2911)	0.052448045385409044
  (0, 2846)	0.052448045385409044
  (0, 2804)	0.052448045385409044
  (0, 2099)	0.04342480267635009
  (0, 216)	0.0339968961953064
  (0, 809)	0.039333118984148235
  (0, 359)	0.041191934356065905
  (0, 2586)	0.02863547061569527
  (0, 222)	0.052448045385409044
  (0, 2640)	0.03298465072953136
  (0, 3091)	0.04996512098303071
  (0, 1799)	0.052448045385409044
  (0, 1647)	0.03702271008966808
  (0, 3028)	0.041191934356065905
  (0, 2520)	0.04996512098303071
  (0, 2951)	0.027999467380609124
  (0, 1787)	0.04996512098303071
  (0, 559)	0.0307958499019654
  (0, 938)	0.044735377737538565
  (0, 1780)	0.055649091678750054
  :	:
  (162, 2614)	0.05039759783263446
  (162, 2901)	0.048056088463964926
  (162, 1439)	0.04420431606554756
  (162, 978)	0.09469223488239202
  (162, 2096)	0.06064910719197336
  (162, 1909)	0.0328

# Latent Semantic Analysis (LSA)

Cosine similarity merupakan metode untuk menghitung nilai kosinus sudut antara vektor dokumen dengan vektor query. Semakin kecil sudut yang dihasilkan, maka tingkat kemiripan esai semakin tinggi.

Formula:

$$
\cos \alpha=\frac{\boldsymbol{A} \cdot \boldsymbol{B}}{|\boldsymbol{A}||\boldsymbol{B}|}=\frac{\sum_{i=1}^{n} \boldsymbol{A}{i} X \boldsymbol{B}{i}}{\sqrt{\sum_{i=1}^{n}\left(\boldsymbol{A}{i}\right)^{2}} X \sqrt{\sum{i=1}^{n}\left(\boldsymbol{B}_{i}\right)^{2}}}
$$

Keterangan:

${A}\> \quad\quad :$ vektor dokumen

${B}\>\quad\quad :$ vektor query

${\boldsymbol{A} \cdot \boldsymbol{B}}\> :$ perkalian dot vektor$

${|\boldsymbol{A}|}\>\quad :$ panjang vektor A

${|\boldsymbol{B}|}\>\quad :$ panjang vektor B$

${|\boldsymbol{A}||\boldsymbol{B}|} :$ Perkalian panjang vektor$

$
\alpha\> \quad\quad :$ sudut yang terbentuk antara vektor A dengan vektor B

In [29]:
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)
lsa=lsa_model.fit_transform(tfidf_res)
lsa

array([[ 2.09215248e-01, -1.75863547e-01, -4.86428961e-01, ...,
         6.60757849e-03, -7.70788342e-02, -6.02521196e-03],
       [ 1.78889933e-01, -7.76807110e-02, -1.43775182e-01, ...,
        -5.04509243e-02,  2.88707491e-01, -3.70327488e-02],
       [ 3.10140228e-01, -2.35939426e-01,  5.60596214e-01, ...,
         2.35984529e-02,  1.89761294e-01,  9.64628607e-02],
       ...,
       [ 1.70229462e-01, -4.59632310e-02, -8.95941252e-02, ...,
         4.41983764e-02, -2.51284933e-04, -9.33293585e-03],
       [ 2.08606937e-01, -5.42626854e-02, -5.58057956e-02, ...,
        -1.77772465e-03, -4.48061162e-03,  3.25683685e-02],
       [ 3.48966366e-01, -1.34400973e-01, -1.14584413e-01, ...,
        -2.26175972e-02,  1.64773288e-02,  1.33358044e-02]])

In [30]:
lsa.shape

(163, 10)

In [31]:
lsa_doc=lsa[0]
for i,t in enumerate(lsa_doc):
  print("Topic ",i," : ",t*100)

Topic  0  :  20.92152480961092
Topic  1  :  -17.586354653598455
Topic  2  :  -48.64289610377172
Topic  3  :  53.59537383387638
Topic  4  :  4.503967070128225
Topic  5  :  12.686528651481405
Topic  6  :  -4.910570513032841
Topic  7  :  0.660757849175592
Topic  8  :  -7.707883420137349
Topic  9  :  -0.6025211956564784


In [32]:
lsa_model.components_

array([[ 8.69836768e-04,  8.48930771e-04,  1.73181631e-02, ...,
         6.05202386e-03,  1.91033363e-03,  8.70402719e-04],
       [-1.74422970e-04, -4.27312186e-04, -6.78157367e-03, ...,
        -5.30722752e-03, -1.12099868e-03, -6.73410279e-04],
       [-1.21660676e-04, -1.09188059e-04, -1.35368717e-02, ...,
         9.16707574e-03,  1.15690151e-05,  7.21401881e-04],
       ...,
       [ 7.35077835e-04, -7.61358774e-04, -8.33337159e-04, ...,
        -2.48009960e-02, -6.26609185e-04, -3.89935271e-03],
       [-2.04363289e-04, -1.55084016e-04, -2.25201575e-04, ...,
        -1.37718496e-03, -1.97893949e-03, -1.91040585e-04],
       [ 2.85684330e-04,  1.83988597e-03, -3.59290785e-04, ...,
        -4.72124457e-02,  1.15756011e-03,  2.88849762e-04]])

In [33]:
lsa_model.components_.shape

(10, 3206)

In [34]:
vocab = tfidf.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
teliti kerja siswa halal ajar uji pengaruh data produk mampu 

Topic 1: 
siswa ajar mampu kritis pikir kelas perangkat guru media persentase 

Topic 2: 
kerja prestasi pegawai garam motivasi karyawan pengaruh efficacy puas uji 

Topic 3: 
undang kerja korupsi pidana hukum berantas sanksi komisi pasal ayat 

Topic 4: 
halal produk kerja sertifikasi ikm literasi lifestyle siswa mui hukum 

Topic 5: 
garam air sp rebon udang bakteri tanam kapal mg stasiun 

Topic 6: 
wisata khas pariwisata pulau giligenting minum kerja makan masyarakat kembang 

Topic 7: 
tanam mangkas pupuk bobot pucuk kandang sapi pakan hst daun 

Topic 8: 
mahasiswa berita papua wacana kritis kerja wisata minum fairclough rusuh 

Topic 9: 
rebon udang kapal mg stasiun air acetes sp sampel kg 



/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
